![DeepLearning.AI logo](https://learn.deeplearning.ai/assets/dlai-logo.png)

# Agentic Workflows

## Reflection Design Pattern

### Learning outcome

Apply reflection patterns to improve agent reasoning and outputs: implement self-evaluation loops where an agent reviews its own intermediate results, identifies potential errors or gaps, and revises its response or tool usage before delivering the final output.


### 🔧 Setup: Import Libraries and Load Environment
This cell loads the necessary Python libraries and reads variables from a .env file to configure the environment.

In [1]:
import pandas as pd
from typing import Dict
from dotenv import load_dotenv
_ = load_dotenv()


### 🤖 Reflection – Agentic AI
This cell initializes the aisuite client, which allows us to interact with different LLMs using a unified interface.

In [2]:
import aisuite as ai

client = ai.Client()

### 🧰 Import utility functions
Import functions to create the sample products database, fetch its schema, and run SQL queries.

In [3]:
from utils import (
    create_transactions_db,
    get_schema,
    execute_sql,
    print_html
)

### 🛠️ Database query workflow
Run this to generate a local SQLite database (products.db) with randomly populated product data for testing queries.

In [4]:
create_transactions_db()

SQLite database 'products.db' created with a single 'transactions' table (event-sourced).


In [5]:
print_html(get_schema('products.db'))

### 🧠 Use an LLM to query a database
This function takes a user question and database schema, then uses an LLM to generate a SQL query that answers the question.

In [6]:
def generate_sql(question: str, schema: str, model: str = "openai:gpt-3.5-turbo") -> str:
    prompt = f"""
You are a SQL assistant. Given the schema and the user's question, write a SQL query for SQLite.

Schema:
{schema}

User question:
{question}

Respond with the SQL only.
"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content.strip()

### 🔍 Reflection with external feedback

This single step first checks whether the SQL result actually answers the user’s question, then—using LLM feedback—automatically proposes a better query if needed. In one pass you get: a brief evaluation, concrete suggestions, and a refined SQL statement aligned with the user’s intent.

In [ ]:
from typing import Tuple

def evaluate_and_refine_sql(
    question: str,
    sql_query: str,
    df: pd.DataFrame,
    schema: str,
    model: str = "openai:gpt-4o",
) -> Tuple[str, str]:
    """
    Evaluate whether the SQL result answers the user's question and,
    if necessary, propose a refined version of the query.
    Returns (feedback, refined_sql).
    """
    prompt = f"""
You are a SQL reviewer and refiner.

User asked:
{question}

Original SQL:
{sql_query}

SQL Output:
{df.to_markdown(index=False)}

Table Schema:
{schema}

Step 1: Briefly evaluate if the SQL output answers the user's question.
Step 2: If the SQL could be improved, provide a refined SQL query.
If the original SQL is already correct, return it unchanged.

Return a strict JSON object with two fields:
- "feedback": brief evaluation and suggestions
- "refined_sql": the final SQL to run
"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )

    import json
    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # Fallback if the model does not return valid JSON:
        # use the raw content as feedback and keep the original SQL
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql


In [8]:
from typing import Any, Dict

def run_sql_workflow(
    db_path: str,
    question: str,
    model_generation: str = "openai:gpt-4.1",
    model_evaluation: str = "openai:gpt-4.1",
) -> Dict[str, Any]:
    """
    End-to-end workflow to generate, execute, evaluate, and refine SQL queries.
    
    Steps:
    1. Extract the database schema.
    2. Generate a candidate SQL query from the user question.
    3. Execute the SQL query and show the initial result.
    4. Evaluate and refine the SQL query (if needed).
    5. Execute the refined query.
    6. Return both the original and refined queries with their results and feedback.
    """
    schema = get_schema(db_path)
    print_html("📘 Get schema:\n" + schema)

    sql = generate_sql(question, schema, model_generation)
    print_html("🧠 Generate SQL (V1):\n" + sql)

    df = execute_sql(sql, db_path)
    print_html("📊 RExecute V1 query → Output:\n" + df.to_html())

    feedback, refined_sql = evaluate_and_refine_sql(
        question=question,
        sql_query=sql,
        df=df,
        schema=schema,
        model=model_evaluation,
    )
    print_html("📝 Reflect on V1 SQL/output:\n" + feedback)
    print_html("🔁 Write V2 query:\n" + refined_sql)

    refined_df = execute_sql(refined_sql, db_path)
    print_html("✅ Execute V2 query → Final answer:\n" + refined_df.to_html())

    return {
        "original_sql": sql,
        "refined_sql": refined_sql,
        "original_result": df,
        "refined_result": refined_df,
        "feedback": feedback,
    }


### ▶️ SQL Workflow Runner
This function ties the full pipeline together: it generates a SQL query, executes it, evaluates the result, refines the SQL using feedback, and shows the final output.

In [9]:
results = run_sql_workflow("products.db", 
                           "Which color of product has the highest total sales?",
                           model_generation="openai:gpt-4.1",
                           model_evaluation="openai:gpt-4.1")

,color,total_sales
0,blue,-190571.46


,color,total_sales
0,white,358315.09
